Retrieve raw PDB files 

In [1]:
from Bio.PDB.PDBList import PDBList
import os

# Initialize PDBList
pdbl = PDBList()

# Directory where the PDB files will be saved
pdb_directory = '/Users/javierherranzdelcerro/Desktop/PYT_SBI/SBPYT_project/raw_pdb'
if not os.path.exists(pdb_directory):
    os.makedirs(pdb_directory)

# Path to your pocket PDB files
pocket_pdb_directory = '/Users/javierherranzdelcerro/Desktop/PYT_SBI/SBPYT_project/dataset/pocket_pdb'

# List to hold the formatted PDB names
pdb_names = []

# Loop through each file in the directory
for filename in os.listdir(pocket_pdb_directory):
    if filename.endswith("_pocket.pdb"):
        # Extract the PDB identifier from the filename
        pdb_id = filename.split("_pocket.pdb")[0]
        
        # Append the formatted name to the list
        pdb_names.append(pdb_id.upper())  # PDB IDs are typically uppercase

for pdb_id in pdb_names:
    # Retrieve PDB file
    pdbl.retrieve_pdb_file(pdb_id, pdir=pdb_directory, file_format='pdb')

    print(f"Downloaded: {pdb_id}")

Downloaded: 5L9G
Downloaded: 1H46
Downloaded: 2PY4
Downloaded: 4JAL
Downloaded: 1NW7
Downloaded: 2P7G
Downloaded: 3QIN
Downloaded: 3GKZ
Downloaded: 1ZZ1
Downloaded: 1DRK
Downloaded: 5HVS
Downloaded: 2V92
Downloaded: 1FKJ
Downloaded: 4DEW
Downloaded: 3BIR
Downloaded: 4W54
Downloaded: 4X1G
Downloaded: 3C9E
Downloaded: 4ZE2
Downloaded: 3B25
Downloaded: 5KR1
Downloaded: 4ALG
Downloaded: 1L5Q
Downloaded: 1YA3
Downloaded: 5ITP
Downloaded: 2F78
Downloaded: 4QJR
Downloaded: 4ZLA
Downloaded: 5FE7
Downloaded: 1CC6
Downloaded: 5LHT
Downloaded: 2QX0
Downloaded: 3VX3
Downloaded: 3HPI
Downloaded: 3T1M
Downloaded: 3F19
Downloaded: 2VC9
Downloaded: 1HS6
Downloaded: 1ZSB
Downloaded: 1UW1
Downloaded: 1RL8
Downloaded: 3DNT
Downloaded: 4F3I
Downloaded: 2UYW
Downloaded: 5KXE
Downloaded: 5E95
Downloaded: 3ARP
Downloaded: 3UJB
Downloaded: 1GX4
Downloaded: 2ZRU
Downloaded: 1K6V
Downloaded: 2IDZ
Downloaded: 4DUF
Downloaded: 1LV8
Downloaded: 1SH9
Downloaded: 4CSD
Downloaded: 2F8I
Downloaded: 3U5L
Downloaded: 4A

In [32]:
pip install scipy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.4/165.4 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 68.4 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


Extract features based on the binding pocket.

In [25]:
from Bio.PDB import PDBParser, NeighborSearch, Selection, is_aa 
import pandas as pd 
import subprocess
from Bio.PDB.MMCIF2Dict import MMCIF2Dict
import os
import numpy as np
import torch
import torch.nn as nn
import os
import numpy as np
from Bio.PDB.DSSP import DSSP
from glob import glob
import hashlib




# Set the working directory
os.chdir('/Users/javierherranzdelcerro/Desktop/PYT_SBI/SBPYT_project')

class ProteinFeatures:
    
    def __init__(self, pdb_file, pocket_pdb_file=None):
        self.dssp_executable = '/opt/homebrew/bin/mkdssp' 
        self.pocket_pdb_file = pocket_pdb_file
        self.pdb_file = pdb_file
        self.parser = PDBParser(QUIET=True)
        self.amino_acids = ['ALA', 'CYS', 'ASP', 'GLU', 'PHE', 'GLY', 'HIS', 'ILE', 'LYS', 'LEU', 
                            'MET', 'ASN', 'PRO', 'GLN', 'ARG', 'SER', 'THR', 'VAL', 'TRP', 'TYR']
        self.secondary_structure_codes = ['H', 'B', 'E', 'G', 'I', 'T', 'S', ' ', '-', 'P']
        self.pocket_residues = self.load_pocket_residues()
        torch.manual_seed(42)  # Set a fixed seed for random number generation
        self.structure = self.parser.get_structure('protein', self.pdb_file)
        self.encoding_ss, self.encoding_aa = self.one_hot_encoding() 

    
    
    def extract_sequence(self):
        parser = PDBParser()
        structure = parser.get_structure('protein', self.pdb_file)
        model = structure[0]  # Assuming single model
        chain = model['A']  # Assuming chain A, change as needed
        residues = chain.get_residues()

        aminoacid_counts = {}
        total_residues = 0

        for residue in residues:
            if residue.get_id()[0] == ' ':
                aminoacid = residue.get_resname()
                aminoacid_counts[aminoacid] = aminoacid_counts.get(aminoacid, 0) + 1
                total_residues += 1

        aminoacid_frequencies = {aminoacid: count / total_residues for aminoacid, count in aminoacid_counts.items()}

        return aminoacid_frequencies
    
    def calculate_total_contact(self):
        """Calculate the total contact for each residue based on a distance threshold."""
        atom_list = Selection.unfold_entities(self.structure, 'A')  # Use self.structure
        ns = NeighborSearch(atom_list)
        total_contact_dict = {}

        for chain in self.structure.get_chains():  # Use self.structure
            for residue in chain:
                if not is_aa(residue, standard=True):  # Skip non-amino acid entities, ensure to import is_aa
                    continue
                residue_key = (chain.id, residue.resname)
                try:
                    alpha_carbon = residue['CA']  # Assuming the alpha carbon is labeled as 'CA'
                    contacts = ns.search(alpha_carbon.get_coord(), 5.0, level='A')  # Search within 10Å radius
                    total_contact_dict[residue_key] = len(contacts) - 1  # Subtract 1 to eX_train_train_train_train_train_train_train_train_train_trainclude the residue itself
                except KeyError:
                    total_contact_dict[residue_key] = 0  # Set total contact to 0 if alpha carbon is missing

        return total_contact_dict


    
    def extract_secondary_structure(self):
        parser = PDBParser(QUIET=True)
        structure = parser.get_structure('protein', self.pdb_file)
        model = structure[0]

        # Generate DSSP data using Biopython's DSSP wrapper
        dssp = DSSP(model, self.pdb_file, dssp=self.dssp_executable)

        # Structured data to store the features
        structured_results = {}

        # Iterate over DSSP output to populate structured_results
        for key in dssp.keys():
            res_id = (key[0], key[1][1])  # Chain ID and residue number (ignoring insertion code for simplicity)
            _, _, ss, access, phi, psi = dssp[key][:6]  # Extract the necessary data
            structured_results[res_id] = {
                'secondary_structure': ss,
                'solvent_accessibility': access,
                'phi': phi,
                'psi': psi
            }

        return structured_results
    
    def load_pocket_residues(self):
        pocket_residues_set = set()
        if self.pocket_pdb_file:
            with open(self.pocket_pdb_file, 'r') as file:
                for line in file:
                    if line.startswith('ATOM'):
                        chain = line[21]
                        residue = line[22:26].strip()
                        pocket_residues_set.add((chain, residue))
        return pocket_residues_set
    
    def one_hot_encoding(self):
        encoding_ss = {}
        encoding_aa = {}
        
        # Encode secondary structure
        for i, code in enumerate(self.secondary_structure_codes):
            one_hot_vector_ss = torch.zeros(len(self.secondary_structure_codes))
            one_hot_vector_ss[i] = 1
            encoding_ss[code] = one_hot_vector_ss
        
        # Encode amino acids
        for i, amino_acid in enumerate(self.amino_acids):
            one_hot_vector_aa = torch.zeros(len(self.amino_acids))
            one_hot_vector_aa[i] = 1 
            encoding_aa[amino_acid] = one_hot_vector_aa
        
        return encoding_ss, encoding_aa
    
    def pdb_id_to_numeric(self, pdb_id):
        """Converts a PDB ID to a numeric value using hashing."""
        # This is a simple way to convert a string to a unique integer
        hash_object = hashlib.sha256(pdb_id.encode())
        # Take the first 8 characters of the hash and convert them to an integer
        hash_hex = hash_object.hexdigest()[:8]
        hash_int = int(hash_hex, 16)
        return hash_int
    
    def extract_features(self):
        structure = self.parser.get_structure('protein', self.pdb_file)
        model = structure[0]  # Asumiendo un único modelo para simplificar
        # Inicializar listas para mantener las características extraídas
        dssp_data = self.extract_secondary_structure()
        all_features = []
        for chain in model.get_chains():
            for residue in chain.get_residues():
                if residue.get_id()[0] == ' ':  # Solo residuos estándar
                    residue_id = residue.get_id()
                    pdb_id = self.pdb_file.split('/')[-1].split('.')[0]
                    pdb_id_numeric = self.pdb_id_to_numeric(pdb_id)
                    residue_name = residue.get_resname()
                    secondary_structure = dssp_data.get((chain.id, residue_id[1]), {}).get('secondary_structure', ' ')
                    solvent_accessibility = dssp_data.get((chain.id, residue_id[1]), {}).get('solvent_accessibility', 0)
                    solvent_accessibility = float(solvent_accessibility) if solvent_accessibility else 0
                    psi_angle = dssp_data.get((chain.id, residue_id[1]), {}).get('psi', np.nan)
                    phi_angle = dssp_data.get((chain.id, residue_id[1]), {}).get('phi', np.nan)
                    In_pocket = int((chain.id, str(residue_id[1])) in self.pocket_residues)
                    total_contact = self.calculate_total_contact().get((chain.id, residue.resname), 0)
                    amino_acid_one_hot = self.encoding_aa[residue_name]
                    secondary_structure_one_hot = [self.encoding_ss[code] for code in secondary_structure]
        
                    feature_dict = {
                        'PDB_ID': pdb_id,
                        'Residue_Name': amino_acid_one_hot,
                        'In_Pocket': In_pocket,
                        'Secondary_structure': secondary_structure_one_hot,
                        'Solvent_accessibility': solvent_accessibility,
                        'Psi_angle': psi_angle,
                        'Phi_angle': phi_angle,
                        'Total_contact': total_contact,
                    }
                    
                    all_features.append(feature_dict)
                    
        return all_features


def featurizer(test_matched_files):
    all_features_list = []
    for pdb_file, pocket_pdb_file in test_matched_files:
        pf = ProteinFeatures(pdb_file, pocket_pdb_file)
        features_list = pf.extract_features()  # This is a list of dictionaries
        all_features_list.extend(features_list)  # Append features for the current PDB file to the list
    return all_features_list



# Example of how to call the matched_pdb_files function
# pdb_dir = '/Users/javierherranzdelcerro/Desktop/PYT_SBI/SBPYT_project/raw_pdb'
# pocket_dir = '/Users/javierherranzdelcerro/Desktop/PYT_SBI/SBPYT_project/dataset/pocket_pdb'
# matched_pdb_files, matched_pocket_files = matched_pdb_files(pdb_dir, pocket_dir)
# matched_files = list(zip(matched_pdb_files, matched_pocket_files))[:1]
# features_dict = extract_features_for_matched_pdb_files(matched_files)
# print(features_dict)




In [26]:
import pickle
from sklearn.model_selection import train_test_split
import os
import glob

def find_matched_pdb_files(pdb_dir, pocket_dir):
    # List all .pdb files in the PDB folder, remove the extension and the prefix
    pdb_files = [os.path.basename(f) for f in glob.glob(os.path.join(pdb_dir, '*.pdb'))]
    pdb_ids = [f[3:-4] for f in pdb_files if f.startswith('pdb') and f.endswith('.pdb')]

    # List all pocket files and remove the extension
    pocket_files = [os.path.basename(f) for f in glob.glob(os.path.join(pocket_dir, '*_pocket.pdb'))]
    pocket_ids = [f[:-11] for f in pocket_files if f.endswith('_pocket.pdb')]

    # Find the intersection of the two sets to ensure each PDB has its pocket
    matched_ids = set(pdb_ids).intersection(set(pocket_ids))

    # Construct the full filenames for the matched files
    matched_pdb_files = [os.path.join(pdb_dir, f'pdb{id}.pdb') for id in matched_ids]
    matched_pocket_files = [os.path.join(pocket_dir, f'{id}_pocket.pdb') for id in matched_ids]

    return matched_pdb_files, matched_pocket_files

def split_data(matched_files, test_size=0.2, val_size=0.25):
    """
    Splits matched PDB files into training, validation, and test sets.
    :param matched_files: List of tuples, each containing matched PDB file paths.
    :param test_size: Fraction of the dataset to include in the test split.
    :param val_size: Fraction of the training dataset to include in the validation split.
    :return: Three lists of tuples: train_files, val_files, test_files.
    """
    # Split into training + validation and test sets
    train_val_files, test_files = train_test_split(matched_files, test_size=test_size, random_state=42)

    # Adjust the size of the validation set relative to the size of the training set
    adjusted_val_size = val_size / (1 - test_size)

    # Split the remaining data into training and validation sets
    train_files, val_files = train_test_split(train_val_files, test_size=adjusted_val_size, random_state=42)

    return train_files, val_files, test_files


# Example of how to call the matched_pdb_files function
pdb_dir = '/Users/javierherranzdelcerro/Desktop/PYT_SBI/SBPYT_project/raw_pdb'
pocket_dir = '/Users/javierherranzdelcerro/Desktop/PYT_SBI/SBPYT_project/dataset/pocket_pdb'
matched_pdb_files_list, matched_pocket_files_list = find_matched_pdb_files(pdb_dir, pocket_dir)
matched_files = list(zip(matched_pdb_files_list, matched_pocket_files_list))[:1000]
train_files, val_files, test_files = split_data(matched_files)

# Saving with Pickle
def save_features_pickle(data, filepath):
    with open(filepath, 'wb') as f:
        pickle.dump(data, f)

# Loading with Pickle
def load_features_pickle(filepath):
    with open(filepath, 'rb') as f:
        return pickle.load(f)

len(train_files), len(val_files), len(test_files)


(550, 250, 200)

In [27]:
features_dict_final_train = featurizer(train_files)
save_features_pickle(features_dict_final_train, 'train_features_final.pkl')
print(features_dict_final_train)


In [20]:
features_dict5_val = featurizer(val_files)
save_features_pickle(features_dict5_val, 'val_features_5.pkl')
print(features_dict5_val)


[{'PDB_ID': 'pdb2q88', 'Residue_Name': tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0.]), 'In_Pocket': 1, 'Secondary_structure': [tensor([0., 0., 0., 0., 0., 0., 0., 0., 1., 0.])], 'Solvent_accessibility': 0.6008064516129032, 'Psi_angle': -27.0, 'Phi_angle': 360.0, 'Total_contact': 32}, {'PDB_ID': 'pdb2q88', 'Residue_Name': tensor([0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.]), 'In_Pocket': 0, 'Secondary_structure': [tensor([0., 0., 0., 0., 0., 0., 0., 0., 1., 0.])], 'Solvent_accessibility': 0.4785276073619632, 'Psi_angle': -45.0, 'Phi_angle': -82.9, 'Total_contact': 31}, {'PDB_ID': 'pdb2q88', 'Residue_Name': tensor([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.]), 'In_Pocket': 0, 'Secondary_structure': [tensor([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.])], 'Solvent_accessibility': 0.5515463917525774, 'Psi_angle': -25.7, 'Phi_angle': -73.1, 'Total_contact': 26

In [21]:
features_dict4_test = featurizer(test_files)
save_features_pickle(features_dict4_test, 'test_features_4.pkl')
print(features_dict4_test)

[{'PDB_ID': 'pdb3lk0', 'Residue_Name': tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.]), 'In_Pocket': 0, 'Secondary_structure': [tensor([0., 0., 0., 0., 0., 0., 0., 0., 1., 0.])], 'Solvent_accessibility': 0.5851063829787234, 'Psi_angle': 147.7, 'Phi_angle': 360.0, 'Total_contact': 24}, {'PDB_ID': 'pdb3lk0', 'Residue_Name': tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0.]), 'In_Pocket': 0, 'Secondary_structure': [tensor([0., 0., 0., 0., 0., 0., 0., 0., 1., 0.])], 'Solvent_accessibility': 0.14615384615384616, 'Psi_angle': 167.3, 'Phi_angle': -81.9, 'Total_contact': 23}, {'PDB_ID': 'pdb3lk0', 'Residue_Name': tensor([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.]), 'In_Pocket': 1, 'Secondary_structure': [tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])], 'Solvent_accessibility': 0.34536082474226804, 'Psi_angle': -52.8, 'Phi_angle': -45.7, 'Total_contact': 

In [71]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import pickle

# Load the features from the pickle file
train_features_filepath = '/Users/javierherranzdelcerro/Desktop/PYT_SBI/SBPYT_project/train_features.pkl'
with open(train_features_filepath, 'rb') as f:
    features_dict_train = pickle.load(f)

print(f"Loaded {len(features_dict_train)} training features")

Loaded 550 training features


In [22]:
import torch
from sklearn.model_selection import train_test_split
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import pickle
from torch.optim.lr_scheduler import StepLR


# Asumiendo que features_array es un array de NumPy con tus características extraídas,
# pdb_names es una lista de identificadores PDB correspondientes,
# y affinities es un array de NumPy con la afinidad de cada complejo.
with open('train_features_11.pkl', 'rb') as f:
    train_features = pickle.load(f)
with open('test_features_4.pkl', 'rb') as g:
    test_features = pickle.load(g)
with open('val_features_5.pkl', 'rb') as h:
    val_features = pickle.load(h)

#df = pd.read_csv('/Users/allalelhommad/PYT/SBPYT_project/sample_dataframe.csv')
# Convert train and test data into pandas dataframes
df_train = pd.DataFrame(train_features).transpose()
df_test = pd.DataFrame(test_features).transpose()
df_val = pd.DataFrame(val_features).transpose()

#Set index
df_test.set_index('PDB_ID', inplace=True)
df_train.set_index('PDB_ID', inplace=True)
df_val.set_index('PDB_ID', inplace=True)

# # Fill the NA values
df_train['Psi_angle'].fillna(df_train['Psi_angle'].mean(), inplace=True)
df_train['Phi_angle'].fillna(df_train['Phi_angle'].mean(), inplace=True)

df_test['Psi_angle'].fillna(df_test['Psi_angle'].mean(), inplace=True)
df_test['Phi_angle'].fillna(df_test['Phi_angle'].mean(), inplace=True)

df_val['Psi_angle'].fillna(df_val['Psi_angle'].mean(), inplace=True)
df_val['Phi_angle'].fillna(df_val['Phi_angle'].mean(), inplace=True)

# Drop the 'PDB_ID' column
X_train = df_train.drop(columns='In_Pocket')
X_test = df_test.drop(columns='In_Pocket')
X_val = df_val.drop(columns='In_Pocket')

# Convert columns to numeric
cols_to_convert = ['Psi_angle', 'Phi_angle', 'Total_contact','Solvent_accessibility']
X_test[cols_to_convert] = X_test[cols_to_convert].apply(pd.to_numeric, errors='coerce')
X_train[cols_to_convert] = X_train[cols_to_convert].apply(pd.to_numeric, errors='coerce')
X_val[cols_to_convert] = X_val[cols_to_convert].apply(pd.to_numeric, errors='coerce')


# Convertir cada lista en 'Residue_Name' en un tensor de PyTorch
X_train['Residue_Name'] = X_train['Residue_Name'].apply(lambda x: x.clone().detach())
X_train['Secondary_structure'] = X_train['Secondary_structure'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else None)
X_test['Residue_Name'] = X_test['Residue_Name'].apply(lambda x: x.clone().detach())
X_test['Secondary_structure'] = X_test['Secondary_structure'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else None)
X_val['Residue_Name'] = X_val['Residue_Name'].apply(lambda x: x.clone().detach())
X_val['Secondary_structure'] = X_val['Secondary_structure'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else None)


# Eliminar las filas con elementos no numéricos en 'Secondary_structure'
X_train = X_train[X_train['Secondary_structure'].notna()]
X_test = X_test[X_test['Secondary_structure'].notna()]
X_val = X_val[X_val['Secondary_structure'].notna()]


# Apilar todos los tensores en 'Secondary_structure' en un solo tensor
secondary_structure_tensor_train = torch.stack(X_train['Secondary_structure'].tolist())
residue_name_tensor_train = torch.stack(X_train['Residue_Name'].tolist())
secondary_structure_tensor_test = torch.stack(X_test['Secondary_structure'].tolist())
residue_name_tensor_test = torch.stack(X_test['Residue_Name'].tolist())
secondary_structure_tensor_val = torch.stack(X_val['Secondary_structure'].tolist())
residue_name_tensor_val = torch.stack(X_val['Residue_Name'].tolist())


# Convertir los datos numéricos a tensores
numeric_data_train = X_train[cols_to_convert].values  # cols_to_convert es tu lista de columnas numéricas
numeric_tensor_train = torch.tensor(numeric_data_train, dtype=torch.float)
numeric_data_test = X_test[cols_to_convert].values
numeric_tensor_test = torch.tensor(numeric_data_test, dtype=torch.float)
numeric_data_val = X_val[cols_to_convert].values
numeric_tensor_val = torch.tensor(numeric_data_val, dtype=torch.float)

# Asegurarse de que todos los tensores tienen la misma longitud en la dimensión 0
assert numeric_tensor_train.shape[0] == secondary_structure_tensor_train.shape[0] == residue_name_tensor_train.shape[0]
assert numeric_tensor_test.shape[0] == secondary_structure_tensor_test.shape[0] == residue_name_tensor_test.shape[0]
assert numeric_tensor_val.shape[0] == secondary_structure_tensor_val.shape[0] == residue_name_tensor_val.shape[0]

# Concatenar los tensores a lo largo de la dimensión 1
X_tensor_train = torch.cat((numeric_tensor_train, secondary_structure_tensor_train, residue_name_tensor_train), dim=1)
X_tensor_test = torch.cat((numeric_tensor_test, secondary_structure_tensor_test, residue_name_tensor_test), dim=1)
X_tensor_val = torch.cat((numeric_tensor_val, secondary_structure_tensor_val, residue_name_tensor_val), dim=1)

# Convert the data to PyTorch tensors
y_train = df_train['In_Pocket']
y_train = y_train.astype(int)  # Convert to integer dtype
y_tensor_train = torch.tensor(y_train.values, dtype=torch.float32)
y_test = df_test['In_Pocket']
y_test = y_test.astype(int)  # Convert to integer dtype
y_tensor_test = torch.tensor(y_test.values, dtype=torch.float32)
y_val = df_val['In_Pocket']
y_val = y_val.astype(int)  # Convert to integer dtype
y_tensor_val = torch.tensor(y_val.values, dtype=torch.float32)

# Define your dataloaders for training and validation
train_dataset = TensorDataset(X_tensor_train, y_tensor_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataset = TensorDataset(X_tensor_val, y_tensor_val)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True)


class MyModel(nn.Module):
    def __init__(self, input_size):
        super(MyModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)  # Output layer, assuming binary classification

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)  # No activation function for the output layer
        return x

# Initialize your model
input_size = X_train.shape[1]
model = MyModel(input_size)

# Define your loss function and optimizer
criterion = nn.BCEWithLogitsLoss()  # Binary cross-entropy loss for binary classification
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer with learning rate 0.001

# Define your optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Define the initial learning rate

# Define your learning rate scheduler
scheduler = StepLR(optimizer, step_size=10, gamma=0.1)  # Reduce the learning rate by half every 10 epochs

# Training loop
num_epochs = 30
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    train_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()  # Zero the gradients
        outputs = model(inputs)  # Forward pass
        labels = labels.view(-1, 1)
        loss = criterion(outputs.squeeze(), labels.squeeze())  # Calculate the loss
        loss.backward()  # Backward pass
        optimizer.step()  # Update weights
        train_loss += loss.item() * inputs.size(0)
    train_loss /= len(train_loader.dataset)
    
    # Validation loop
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0
    correct = 0
    total = 0  # Initialize total to 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            outputs = outputs.squeeze()
            loss = criterion(outputs, labels)  # Calculate the loss
            val_loss += loss.item() * inputs.size(0)
            predicted = torch.round(torch.sigmoid(outputs))  # Round to get binary predictions
            correct += torch.eq(predicted, labels.view_as(predicted)).sum().item()  # Compare predicted with labels
            total += labels.size(0)  # Increment total by batch size

    val_loss /= total
    val_accuracy = correct / total

    
    # Print training/validation statistics
    print(f'Epoch {epoch + 1}/{num_epochs}, Training Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2%}')

# Evaluate the model on the test set
# Set the model to evaluation mode
model.eval()

# Disable gradient calculation
with torch.no_grad():
    # Forward pass to get the outputs
    test_outputs = model(X_test)
    
    # Calculate the loss
    test_loss = criterion(test_outputs.squeeze(), y_test)
    
    # Apply sigmoid activation to the outputs and round to get binary predictions
    predicted_test = torch.round(torch.sigmoid(test_outputs))
    
    # Calculate the number of correct predictions
    correct_predictions = (predicted_test == y_test.unsqueeze(1)).sum().item()
    
    # Calculate the total number of samples in the test set
    total_samples = len(y_test)
    
    # Calculate the test accuracy
    test_accuracy = correct_predictions / total_samples
    
    # Print test loss and accuracy
    print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2%}')


KeyError: "None of ['PDB_ID'] are in the columns"

In [ ]:

df.set_index('PDB_ID', inplace=True)
print(df.head())
# Drop the 'PDB_ID' column
X = df.drop(columns='In_Pocket')
X_numeric = X.select_dtypes(include=['float64', 'float32', 'float16', 'int64', 'int32', 'int16', 'int8'])
y = df['In_Pocket']

# Convert the data to PyTorch tensors
X_tensor = torch.tensor(X_numeric.values, dtype=torch.float32)
y_tensor = torch.tensor(y.values, dtype=torch.float32)

# Split the data into training and validation sets

X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.33, random_state=42)



# Define your deep learning model


class MyModel(nn.Module):
    def __init__(self, input_size):
        super(MyModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)  # Output layer, assuming binary classification

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)  # No activation function for the output layer
        return x

# Initialize your model
input_size = X_train.shape[1]
model = MyModel(input_size)

# Define your loss function and optimizer
criterion = nn.BCEWithLogitsLoss()  # Binary cross-entropy loss for binary classification
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer with learning rate 0.001

# Define your dataloaders for training and validation
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataset = TensorDataset(X_val, y_val)
val_loader = DataLoader(val_dataset, batch_size=32)


# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    train_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()  # Zero the gradients
        outputs = model(inputs)  # Forward pass
        loss = criterion(outputs.squeeze(), labels)  # Calculate the loss
        loss.backward()  # Backward pass
        optimizer.step()  # Update weights
        train_loss += loss.item() * inputs.size(0)
    train_loss /= len(train_loader.dataset)
    
    # Validation loop
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0
    correct = 0
    total = 0  # Initialize total to 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), labels)
            val_loss += loss.item() * inputs.size(0)
            predicted = torch.round(torch.sigmoid(outputs))  # Round to get binary predictions
            correct += torch.eq(predicted, labels.view_as(predicted)).sum().item()  # Compare predicted with labels
            total += labels.size(0)  # Increment total by batch size

    val_loss /= total
    val_accuracy = correct / total

    
    # Print training/validation statistics
    print(f'Epoch {epoch + 1}/{num_epochs}, Training Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2%}')

# Evaluate the model on the test set
# Set the model to evaluation mode
model.eval()

# Disable gradient calculation
with torch.no_grad():
    # Forward pass to get the outputs
    test_outputs = model(X_test)
    
    # Calculate the loss
    test_loss = criterion(test_outputs.squeeze(), y_test)
    
    # Apply sigmoid activation to the outputs and round to get binary predictions
    predicted_test = torch.round(torch.sigmoid(test_outputs))
    
    # Calculate the number of correct predictions
    correct_predictions = (predicted_test == y_test.unsqueeze(1)).sum().item()
    
    # Calculate the total number of samples in the test set
    total_samples = len(y_test)
    
    # Calculate the test accuracy
    test_accuracy = correct_predictions / total_samples
    
    # Print test loss and accuracy
    print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2%}')


Prepare the Data

In [40]:
import torch
from sklearn.model_selection import train_test_split
import numpy as np
import torch.nn as nn
import torch.optim as optim
from feature_extraction import features
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd

# Asumiendo que features_array es un array de NumPy con tus características extraídas,
# pdb_names es una lista de identificadores PDB correspondientes,
# y affinities es un array de NumPy con la afinidad de cada complejo.
pdb_files=['dataset/1x8v.pdb','dataset/182l.pdb']
pocket_pdb_files=['dataset/pocket_pdb/1x8v_pocket.pdb','dataset/pocket_pdb/182l_pocket.pdb']

# Call the features function to get the extracted features
extracted_features = features(pdb_files, pocket_pdb_files)

df = pd.DataFrame(extracted_features)
# Extract PDB IDs
pdb_ids = df['PDB_ID']

# Drop the 'PDB_ID' column
X = df.drop(columns=['In_Pocket', 'PDB_ID'])
X_numeric = X.select_dtypes(include=['float64', 'float32', 'float16', 'int64', 'int32', 'int16', 'int8'])
print(X_numeric)
y = df['In_Pocket']

# Convert the data to PyTorch tensors
X_tensor = torch.tensor(X_numeric.values, dtype=torch.float32)
y_tensor = torch.tensor(y.values, dtype=torch.float32)

# Shuffle the data
indices = torch.randperm(X_tensor.size(0))
X_tensor = X_tensor[indices]
y_tensor = y_tensor[indices]
pdb_ids_shuffled = pdb_ids.iloc[indices]

# Split the data into training/validation/testing sets (70% train, 10% validation, 20% test)
X_train_val, X_test, y_train_val, y_test, pdb_ids_train_val, pdb_ids_test = train_test_split(
    X_tensor, y_tensor, pdb_ids_shuffled, test_size=0.3, random_state=42)
X_train, X_val, y_train, y_val, pdb_ids_train, pdb_ids_val = train_test_split(
    X_train_val, y_train_val, pdb_ids_train_val, test_size=0.25, random_state=42)

X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

# Create DataLoader for training, validation, and test sets
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64)
test_loader = DataLoader(test_dataset, batch_size=64)

class MyModel(nn.Module):
    def __init__(self, input_size):
        super(MyModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x

# Instantiate the model
input_size = len(X_train.columns)
model = MyModel(input_size)

# Define loss function and optimizer
criterion = nn.BCELoss()  # Binary cross-entropy loss for binary classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 10
for epoch in range(epochs):
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels.unsqueeze(1))
        loss.backward()
        optimizer.step()
    
    # Validation loop
    model.eval()
    with torch.no_grad():
        val_loss = 0.0
        for inputs, labels in val_loader:
            outputs = model(inputs)
            val_loss += criterion(outputs, labels.unsqueeze(1)).item()
        val_loss /= len(val_loader)
    
    print(f'Epoch {epoch+1}/{epochs}, Train Loss: {loss.item():.4f}, Val Loss: {val_loss:.4f}')

# Evaluate on test set
model.eval()
with torch.no_grad():
    test_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in test_loader:
        outputs = model(inputs)
        test_loss += criterion(outputs, labels.unsqueeze(1)).item()
        predicted = torch.round(outputs)
        total += labels.size(0)
        correct += (predicted == labels.unsqueeze(1)).sum().item()

print(f'Test Loss: {test_loss/len(test_loader):.4f}, Test Accuracy: {100 * correct / total:.2f}%')


ModuleNotFoundError: No module named 'sklearn'

Define and Train the Model

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import h5py
from sklearn.model_selection import train_test_split
from utils import progress_bar

# Model Definition
class BindingPocketCNN(nn.Module):
    def __init__(self, num_features, seq_length):
        super(BindingPocketCNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=num_features, out_channels=64, kernel_size=5, padding=2)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=5, padding=2)
        # Adjust the definition of fc1 according to the sequence length adjusted by the pooling layers
        self.fc1 = nn.Linear(128 * (seq_length // 4), 128)
        self.fc2 = nn.Linear(128, 1)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)  # Flatten all dimensions except for the batch
        x = F.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x

# Load Data
def load_data(input_path):
    with h5py.File(f'{input_path}/training_set.hdf5', 'r') as f:
        X_train = torch.tensor(f['features'][:], dtype=torch.float)
        y_train = torch.tensor(f['affinities'][:], dtype=torch.float).view(-1, 1)
    return X_train, y_train

# Data Preparation
input_path = '../dataset'  # Adjust as necessary
X_train, y_train = load_data(input_path)
seq_length = X_train.shape[2]  # Assuming features are in dimension 2
num_features = X_train.shape[1]

# Data Splitting (assuming data are already loaded and prepared)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Model Initialization and Training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BindingPocketCNN(num_features=num_features, seq_length=seq_length).to(device)

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 100  # Adjust as necessary

for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        progress_bar(i, len(train_loader), 'Loss: %.3f' % (loss.item()))
    
    # Validation step (optional, but recommended)
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            val_loss += criterion(outputs, labels).item()
            progress_bar(i, len(val_loader), 'Val Loss: %.3f' % (val_loss/(i+1)))

    print(f'Epoch {epoch+1}, Train Loss: {loss.item():.4f}, Validation Loss: {val_loss/len(val_loader):.4f}')


ModuleNotFoundError: No module named 'h5py'